In [1]:
import torch
import matplotlib.pyplot as plt
import os
import numpy as np
import copy

In [2]:
results = torch.load('flow_complexity_results_n={0}_trials={1}_empiricalpx.pt'.format(1000,50))
results_flow = torch.load('causalflow_complexity_results_n={0}_trials={1}.pt'.format(1000,50))

In [3]:
list(results[0].keys())[:-1], list(results_flow[0].keys())[:2]

(['L2', 'L1', 'URR L2', 'URR L1', 'HSIC', 'CMMD_V', 'CMMD_U'],
 ['GaussianFlow', 'LaplaceFlow'])

In [4]:
nmodel = 9
model_select = [0,1,2,3,4,5,7,8]
model_keys = list(results[0].keys())[:-1]
flow_keys = list(results_flow[0].keys())[:2]
seeds = 50
noise_dists = 5
KSDs = torch.zeros((nmodel, seeds, noise_dists))
RMSEs = torch.zeros((nmodel, seeds, noise_dists))
Indexes = torch.zeros((nmodel, seeds, noise_dists))

In [5]:
r = 0
for i in range(seeds):
    for n in range(noise_dists):
        for m in range(len(flow_keys)):
            KSDs[m,i,n] = results_flow[r][flow_keys[m]]['KS_int']
            RMSEs[m,i,n] = results_flow[r][flow_keys[m]]['CF_RMSE']
            Indexes[m,i,n] = results_flow[r][flow_keys[m]]['index']
        r += 1

r = 0
for i in range(seeds):
    for n in range(noise_dists):
        for m in range(len(model_keys)):
            KSDs[m+len(flow_keys),i,n] = results[r][model_keys[m]]['KS_int']
            RMSEs[m+len(flow_keys),i,n] = results[r][model_keys[m]]['CF_RMSE']
            Indexes[m+len(flow_keys),i,n] = results[r][model_keys[m]]['index']
        r += 1

KSDs = KSDs[model_select]
RMSEs = RMSEs[model_select]
Indexes = Indexes[model_select]

In [6]:
good_inds = torch.where(RMSEs[-1].sum(1) < 10)
RMSEs[-1][good_inds].mean(0),RMSEs[-1][good_inds].std(0)

(tensor([0.0449, 0.0294, 0.0448, 0.0927, 0.0160]),
 tensor([0.0503, 0.0455, 0.0613, 0.4547, 0.0112]))

In [7]:
torch.set_printoptions(sci_mode=False)
KSDs.mean(1),RMSEs.mean(1),(Indexes == 0).float().mean(1)

(tensor([[0.0193, 0.0371, 0.1840, 0.2444, 0.0453],
         [0.0505, 0.0503, 0.0693, 0.0749, 0.0736],
         [0.0342, 0.0429, 0.1170, 0.1429, 0.0832],
         [0.0383, 0.0415, 0.0692, 0.0802, 0.0940],
         [0.0371, 0.0538, 0.0686, 0.0835, 0.0508],
         [0.0538, 0.0608, 0.0552, 0.0935, 0.0519],
         [0.0312, 0.0307, 0.0288, 0.0257, 0.0288],
         [0.0306, 0.0314, 0.0276, 0.0274, 0.0278]]),
 tensor([[    0.0387,     0.2619,    86.7313,   137.3304,     0.3850],
         [    0.1900,     0.2095,   100.6141,   115.9098,     0.3218],
         [    0.0398,     0.2235,    49.0776,    63.2286,     0.2363],
         [    0.0451,     0.1771,    77.8780,    88.6083,     0.2927],
         [    0.0436,     0.1322,    10.0235,    15.1100,     0.1174],
         [    0.0909,     0.1585,    11.7627,    16.3792,     0.1410],
         [    0.0383,     0.0252,     0.2476,     0.5220,     0.0193],
         [    0.0456,     0.0291,     0.0439,     1.4050,     2.2022]]),
 tensor([[0.9400, 0.

In [11]:
KSDs.std(1),RMSEs.std(1),(Indexes == 0).float().std(1)

(tensor([[0.0087, 0.0226, 0.1583, 0.1945, 0.0198],
         [0.0195, 0.0217, 0.0338, 0.0537, 0.0792],
         [0.0115, 0.0151, 0.1057, 0.1099, 0.1082],
         [0.0119, 0.0161, 0.0266, 0.0577, 0.0710],
         [0.0116, 0.0094, 0.0097, 0.0084, 0.0098],
         [0.0097, 0.0110, 0.0104, 0.0107, 0.0102]]),
 tensor([[    0.0451,     0.0723,   284.1096,   254.4824,     0.3303],
         [    0.1237,     0.0499,   348.2874,   210.5683,     0.3174],
         [    0.0480,     0.0695,   190.8556,   116.1224,     0.3512],
         [    0.0451,     0.0936,   286.9599,   177.2745,     0.3846],
         [    0.0399,     0.0185,     1.1036,     3.4609,     0.0246],
         [    0.0489,     0.0446,     0.0597,     7.3934,    15.4540]]),
 tensor([[0.2399, 0.1414, 0.0000, 0.0000, 0.0000],
         [0.4712, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1979, 0.1414, 0.0000, 0.0000, 0.0000],
         [0.1979, 0.2740, 0.0000, 0.1414, 0.1979],
         [0.1979, 0.0000, 0.2399, 0.2399, 0.2399],
         

In [34]:
import numpy as np

# --- replace these with your actual data arrays ---
# ksd[m, t, d] = KSD for method m, trial t, noise d
# rmse[m, t, d] = RMSE for method m, trial t, noise d
# idx[m, t, d]  = index of the model selected at trial t, noise d
#               values in 0..(n_methods-1)
ksd  = KSDs.numpy()
rmse = RMSEs.numpy()
idx  = Indexes.numpy()
# ----------------------------------------------------------------

methods = [
    "Abductive Flow (G)",
    "Abductive Flow (L)",
    "Conditional Flow (G)",
    "Conditional Flow (L)",
    "Cocycles (CMMD‑V)",
    "Cocycles (CMMD‑U)",
]
noise_names = [
    r"$\mathcal N(0,1)$",
    r"$Ga(1,1)$",
    r"$t_1(0,1)$",
    r"$IG(1,1)$",
    r"$Rad(\tfrac12)$",
]

M, T, D = ksd.shape

# 1) Compute mean ± std across trials
ksd_mean  = ksd.mean(axis=1)   # (M, D)
ksd_std   = ksd.std(axis=1)
rmse_mean = rmse.mean(axis=1)
rmse_std  = rmse.std(axis=1)

# 2) Percent‐selection of true arch (idx==0)
perc = np.zeros((M, D))
for m in range(M):
    perc[m] = (idx[m] == 0).mean(axis=0)  # mean over trials ⇒ shape (D,)

# 3) Best‐2 indices per column
best2_ksd  = np.argsort(ksd_mean, axis=0)[:2]   # two smallest
best2_rmse = np.argsort(rmse_mean, axis=0)[:2]
best2_perc = np.argsort(-perc,    axis=0)[:2]  # two largest

# 4) Formatting helper (3 s.f.)
def fmt_ms(m, s):
    return f"{m:.3g} $\\pm$ {s:.3g}"

# 5) Emit LaTeX
print(r"\begin{table}[ht]")
print(r"\centering\small")
print(r"\begin{tabular}{l" + "c"*D + r"}")
print(r"\toprule")
print("Method & " + " & ".join(noise_names) + r" \\")
print(r"\midrule")

# Interventional KSD
print(r"\multicolumn{" + f"{D+1}" + r"}{c}{\textbf{Interventional KSD}} \\")
print(r"\cmidrule(lr){1-" + f"{D+1}" + r"}")
for m, name in enumerate(methods):
    cells = []
    for j in range(D):
        cell = fmt_ms(ksd_mean[m,j], ksd_std[m,j])
        if m in best2_ksd[:,j]:
            cell = r"\bfseries " + cell
        cells.append(cell)
    print(name + " & " + " & ".join(cells) + r" \\")
print(r"\midrule")

# Counterfactual RMSE
print(r"\multicolumn{" + f"{D+1}" + r"}{c}{\textbf{Counterfactual RMSE}} \\")
print(r"\cmidrule(lr){1-" + f"{D+1}" + r"}")
for m, name in enumerate(methods):
    cells = []
    for j in range(D):
        cell = fmt_ms(rmse_mean[m,j], rmse_std[m,j])
        if m in best2_rmse[:,j]:
            cell = r"\bfseries " + cell
        cells.append(cell)
    print(name + " & " + " & ".join(cells) + r" \\")
print(r"\midrule")

# True Model Selection %
print(r"\multicolumn{" + f"{D+1}" + r"}{c}{\textbf{True Model Selection \%}} \\")
print(r"\cmidrule(lr){1-" + f"{D+1}" + r"}")
for m, name in enumerate(methods):
    cells = []
    for j in range(D):
        val = perc[m,j]
        cell = f"{val:.3g}"
        if m in best2_perc[:,j]:
            cell = r"\bfseries " + cell
        cells.append(cell)
    print(name + " & " + " & ".join(cells) + r" \\")
print(r"\bottomrule")
print(r"\end{tabular}")
print(r"\caption{Means~$\pm$~stds (3 s.f.) for Interventional KSD, Counterfactual RMSE, and True‐model‐selection \% across five noise distributions. Best two per column bolded.}")
print(r"\end{table}")


\begin{table}[ht]
\centering\small
\begin{tabular}{lccccc}
\toprule
Method & $\mathcal N(0,1)$ & $Ga(1,1)$ & $t_1(0,1)$ & $IG(1,1)$ & $Rad(\tfrac12)$ \\
\midrule
\multicolumn{6}{c}{\textbf{Interventional KSD}} \\
\cmidrule(lr){1-6}
Abductive Flow (G) & \bfseries 0.0193 $\pm$ 0.00863 & 0.0371 $\pm$ 0.0224 & 0.184 $\pm$ 0.157 & 0.244 $\pm$ 0.193 & 0.0453 $\pm$ 0.0196 \\
Abductive Flow (L) & 0.0505 $\pm$ 0.0193 & 0.0503 $\pm$ 0.0215 & 0.0693 $\pm$ 0.0334 & 0.0749 $\pm$ 0.0531 & 0.0736 $\pm$ 0.0784 \\
Conditional Flow (G) & 0.0342 $\pm$ 0.0113 & 0.0429 $\pm$ 0.0149 & 0.117 $\pm$ 0.105 & 0.143 $\pm$ 0.109 & 0.0832 $\pm$ 0.107 \\
Conditional Flow (L) & 0.0383 $\pm$ 0.0118 & 0.0415 $\pm$ 0.0159 & 0.0692 $\pm$ 0.0263 & 0.0802 $\pm$ 0.0571 & 0.094 $\pm$ 0.0703 \\
Cocycles (CMMD‑V) & 0.0312 $\pm$ 0.0115 & \bfseries 0.0307 $\pm$ 0.00927 & \bfseries 0.0288 $\pm$ 0.00964 & \bfseries 0.0257 $\pm$ 0.00828 & \bfseries 0.0288 $\pm$ 0.00973 \\
Cocycles (CMMD‑U) & \bfseries 0.0306 $\pm$ 0.0096 & \bfserie